# 结构方程模型专题


In [1]:
import sys
import os
import cudf  #CUDA计算
import pandas as pd
import numpy as np

# 自行编写的包
sys.path.append('/home/ubuntu/notebook/Investor-Sentiment')
sys.path.append('/usr/local/stata17/utilities')
from loader.findata_loader import Loader
from utils.sql import DB

# Stata
from pystata import config

config.init('mp')
# ------------------------------数据集路径----------------------------------#
DATASETS_PATH = '/data/DataSets/investor_sentiment/'


  ___  ____  ____  ____  ____ ©
 /__    /   ____/   /   ____/      17.0
___/   /   /___/   /   /___/       MP—Parallel Edition

 Statistics and Data Science       Copyright 1985-2021 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-STATA-PC        https://www.stata.com
                                   979-696-4600        stata@stata.com

Stata license: Single-user 8-core , expiring  1 Jan 2025
Serial number: 501709301094
  Licensed to: Colin's Stata
               Love U

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. More than 2 billion observations are allowed; see help obs_advice.
      3. Maximum number of variables is set to 5,000; see help set_maxvar.


## 数据准备
详细的各个指标计算见每个笔记本,这里是已经处理和计算好的数据

### 媒体情绪

In [2]:
df_media = cudf.read_parquet(DATASETS_PATH + 'MEDIA_SENT_TDATE_WC.parquet')
df_media
#

,img_neg,tex_neg
trade_date,,
20140102,0.000000,0.000000
20140103,0.000000,0.333333
20140106,0.285714,0.142857
20140107,0.000000,0.333333
20140108,0.333333,0.000000
...,...,...
20220621,0.238095,0.285714
20220622,0.227273,0.227273
20220623,0.363636,0.363636


### 投资者情绪

In [3]:
df_investor = cudf.read_parquet(DATASETS_PATH + 'FORUM_SENT_TDATE_CSMAR.parquet')
df_investor

,IndexA,IndexB,Posts,Readings,Comments,ThumbUps,PosPosts,NeuPosts,NegPosts,PosRatio,NeuRatio,NegRatio
trade_date,,,,,,,,,,,,
20140102,NaN,0.000000,0.0,<NA>,<NA>,<NA>,0.0,0.0,0.0,NaN,NaN,NaN
20140103,0.000000,0.000000,2.0,107379.0,342.0,2.0,1.0,0.0,1.0,0.500000,0.000000,0.500000
20140106,0.000000,0.000000,2.0,22652.0,47.0,0.0,1.0,0.0,1.0,0.500000,0.000000,0.500000
20140107,NaN,0.000000,0.0,<NA>,<NA>,<NA>,0.0,0.0,0.0,NaN,NaN,NaN
20140108,NaN,0.000000,0.0,<NA>,<NA>,<NA>,0.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
20211012,-0.229368,-0.467035,120497.0,26442508.27,273224.2,172338.87,42169.0,11057.0,67271.0,0.349959,0.091762,0.558279
20211013,-0.226023,-0.459981,123178.0,27651587.71,288526.17,177561.46,43256.0,11402.0,68520.0,0.351167,0.092565,0.556268
20211014,-0.188129,-0.380788,135718.0,30212375.19,303991.93,188185.02,50036.0,12457.0,73225.0,0.368676,0.091786,0.539538


### 分析师情绪

In [6]:
df_analyst = cudf.read_parquet(DATASETS_PATH + 'ANALYST_SENT_TDATE_SUNTIME.parquet')
df_analyst

CON_YEAR        CON_OR        CON_NP  CON_EPS  \
trade_date ts_code                                                    
20140102   000010.SH      2014  1.229380e+09  20139.051805   0.8059   
           000015.SH      2014  5.889377e+08  15328.983195   0.8057   
           000016.SH      2014  8.063073e+08  13294.314326   0.8620   
           000300.SH      2014  1.553438e+09  22802.196329   0.7898   
           000903.SH      2014  1.219844e+09  20305.917391   0.8299   
...                        ...           ...           ...      ...   
20221028   000906.SH      2022  5.610430e+09  51796.277367   0.9641   
           000985.SH      2022  7.224144e+09  58403.840993   0.7555   
           399005.SZ      2022  3.308684e+08   3156.638702   0.9635   
           399006.SZ      2022  1.580034e+08   1800.383211   1.3799   
           399310.SZ      2022  1.525638e+09  15430.058098   1.3643   

                            CON_NA  CON_PB  CON_PS   CON_PE  CON_PEG  CON_ROE  \
trade_date ts_code                                                              
20140102   000010.SH  1.356969e+09  1.0078  0.7662   6.7909   0.5332    14.84   
           000015.SH  9.912300e+08  0.8505  0.8438   5.4999   0.5358    15.46   
           000016.SH  8.877149e+08  0.9876  0.7290   6.5949   0.5269    14.98   
           000300.SH  1.569081e+09  1.0997  0.7828   7.5674   0.5488    14.53   
           000903.SH  1.353838e+09  0.9884  0.7627   6.5896   0.5254    15.00   
...                            ...     ...     ...      ...      ...      ...   
20221028   000906.SH  4.774941e+09  1.1471  0.9763  10.5749   0.7286    10.85   
           000985.SH  5.825681e+09  1.3191  1.0637  13.1574   0.7802    10.03   
           399005.SZ  2.227230e+08  2.8926  1.9471  20.4090   0.5528    14.17   
           399006.SZ  1.220097e+08  4.4799  3.4594  30.3598   0.7268    14.76   
           399310.SZ  1.151031e+09  1.4075  1.0619  10.4997   0.8521    13.41   

                      ...  total_mv_l1  total_mv_chg   close_l1  close_chg  \
trade_date ts_code    ...                                                    
20140102   000010.SH  ...    136823.66     -0.545520  5040.2700  -0.599174   
           000015.SH  ...     85036.43     -0.535747  1681.4600  -0.397274   
           000016.SH  ...     87771.07     -0.634059  1574.7800  -0.868693   
           000300.SH  ...    171737.19     -0.385246  2330.0300  -0.345489   
           000903.SH  ...    133275.65     -0.527471  2155.2300  -0.593904   
...                   ...          ...           ...        ...        ...   
20221028   000906.SH  ...    559898.44     -2.218877  4024.5400  -2.622412   
           000985.SH  ...    828763.95     -2.492601  4640.7167  -2.964490   
           399005.SZ  ...     66542.30     -3.186169  7293.5300  -3.140318   
           399006.SZ  ...     56664.70     -3.546670  2337.2500  -3.711199   
           399310.SZ  ...    166216.95     -2.517926  5682.8900  -2.940229   

                      riskfree_return  opt_profit_yoy     opt_pe  opt_profit  \
trade_date ts_code                                                             
20140102   000010.SH         0.015460      106.321095 -36.175752    6.534466   
           000015.SH         0.015460       19.072326 -30.115629    4.676568   
           000016.SH         0.015460      131.053412 -32.636364    6.312925   
           000300.SH         0.015460       76.005909 -36.833055    7.697893   
           000903.SH         0.015460       80.367172 -35.141732    6.049136   
...                               ...             ...        ...         ...   
20221028   000906.SH         0.004842      203.012356 -18.087529   40.330525   
           000985.SH         0.004842      241.903541 -22.694477   36.381384   
           399005.SZ         0.004842       83.540859 -23.129944   50.151677   
           399006.SZ         0.004842     8845.061854 -24.327517   52.148060   
           399310.SZ         0.004842      202.048409 -17.063981   35